In [168]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version',tf.__version__)

TF version 2.7.0


#### Set up Tokenizer

In [98]:
MAX_LEN = 96
PATH = '../config/'

tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab=PATH+'vocab-roberta-base.json', 
    merges=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}

In [169]:
test.loc[2, 'text']

'Recession hit Veronique Branquinho, she has to quit her company, such a shame!'

### Preprocess Data

In [170]:
# Data preprocessing
test = pd.read_csv('test.csv').fillna('')
tweet = test.loc[2, 'text'] #'Last session of the day  http://twitpic.com/67ezh'
sentiment = 'negative'
ct = 1
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')

# INPUT_IDS
text1 = " "+" ".join(tweet.split())
enc = tokenizer.encode(text1)
s_tok = sentiment_id[sentiment]           
input_ids[0,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
attention_mask[0,:len(enc.ids)+5] = 1

#### Set up Model

In [171]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Conv1D(1,1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1,1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [172]:
model = build_model()
model.load_weights("../weights_final.h5")

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


#### Inference

In [173]:
DISPLAY = 1
preds = model.predict([input_ids, attention_mask, token_type_ids],verbose=DISPLAY)

1/1 [==============================] - 7s 7s/step


In [178]:
preds[0].shape

(1, 96)

In [183]:
preds_start = np.zeros((input_ids.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids.shape[0],MAX_LEN))
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=777)
preds_start += preds[0] #/skf.n_splits
preds_end += preds[1] #/skf.n_splits

In [184]:
len(preds_start)

1

In [143]:
all = []
for k in range(input_ids.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = tweet
    else:
        text1 = " "+" ".join(tweet.split())
        print(text1)
        enc = tokenizer.encode(text1)
        print(enc.ids)
        print(a-1, b)
        st = tokenizer.decode(enc.ids[a-1:b])
    all.append(st)

In [148]:
a = np.argmax(preds_start[0,])
b = np.argmax(preds_end[0,])
if a>b: 
    selected_text = tweet
else:
    text1 = " "+" ".join(tweet.split())
    print(text1)
    enc = tokenizer.encode(text1)
    print(enc.ids)
    print(a-1, b)
    selected_text = tokenizer.decode(enc.ids[a-1:b])


['Today after school, I saw a lost puppy, so sad!']

In [144]:
tokenizer.decode(enc.ids[max(a-1, 0):b])

''

In [ ]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [156]:
test = pd.read_csv('test.csv').fillna('')

ct = test[:10].shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test[:10].shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

In [157]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [158]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=777)
for fold in range(skf.n_splits):#,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
        
    #sv = tf.keras.callbacks.ModelCheckpoint(
    #    '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
    #    save_weights_only=True, mode='auto', save_freq='epoch')
        
    #model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
    #    epochs=3, batch_size=32, verbose=DISPLAY, callbacks=[sv],
    #    validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
    #    [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights("../weights_final.h5") #('%s-roberta-%i.h5'%(VER,fold))
    
    #print('Predicting OOF...')
    #oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    #all = []
    #for k in idxV:
    #    a = np.argmax(oof_start[k,])
    #    b = np.argmax(oof_end[k,])
    #    if a>b: 
    #        st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
    #    else:
    #        text1 = " "+" ".join(train.loc[k,'text'].split())
    #        enc = tokenizer.encode(text1)
    #        st = tokenizer.decode(enc.ids[a-1:b])
    #    all.append(jaccard(st,train.loc[k,'selected_text']))
    #jac.append(np.mean(all))
    #print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    #print()

#########################
### FOLD 1
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
1/1 [==============================] - 8s 8s/step
#########################
### FOLD 2
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
1/1 [==============================] - 8s 8s/step
#########################
### FOLD 3
#########################


...

Loading model...
Predicting Test...
1/1 [==============================] - 10s 10s/step
#########################
### FOLD 4
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
1/1 [==============================] - 7s 7s/step
#########################
### FOLD 5
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Loading model...
Predicting Test...
1/1 [==============================] - 8s 8s/step


In [160]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-1:b])
    all.append(st)

In [163]:
pd.DataFrame({'text': test['text'][:10].values, 'selected_test': all})

,text,selected_test
0,Last session of the day http://twitpic.com/67ezh,itpic.com
1,Shanghai is also really exciting (precisely -...,
2,"Recession hit Veronique Branquinho, she has to...","recession hit veronique branquinho, she has t..."
3,happy bday!,
4,http://twitpic.com/4w75p - I like it!!,http://twitpic.com/4w75
5,that`s great!! weee!! visitors!,
6,I THINK EVERYONE HATES ME ON HERE lol,on here
7,"soooooo wish i could, but im in school and my...","soooooo wish i could, but im in school and my..."
8,and within a short time of the last clue all ...,
9,What did you get? My day is alright.. haven`...,leaving soon to my
